## 1.  Chargement des Données

In [36]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

In [37]:
# Charger les données d'entraînement
df = pd.read_csv('X_train.csv')
y = pd.read_csv('y_train.csv', index_col=0)

print(f"✓ X_train chargé: {df.shape}")
print(f"✓ y_train chargé: {y.shape}")

✓ X_train chargé: (1172086, 307)
✓ y_train chargé: (1172086, 1)


## 2.  Exploration Initiale des Données

In [38]:
# Informations générales sur le dataset
df.shape

(1172086, 307)

In [39]:
# Analyse des valeurs manquantes
print("=== ANALYSE DES VALEURS MANQUANTES ===\n")
missing_percent = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)
print(f"Colonnes avec >60% de NaN: {(missing_percent > 60).sum()}")
print(f"Colonnes avec >20% de NaN: {(missing_percent > 20).sum()}")
print(f"Colonnes sans NaN: {(missing_percent == 0).sum()}")

=== ANALYSE DES VALEURS MANQUANTES ===

Colonnes avec >60% de NaN: 200
Colonnes avec >20% de NaN: 288
Colonnes sans NaN: 14


In [40]:
# Statistiques de la variable cible (MathScore)
print("=== VARIABLE CIBLE: MATHSCORE ===\n")
print(y.describe())
print(f"\nDistribution:")
print(f"  Min: {y['MathScore'].min():.2f}")
print(f"  25%: {y['MathScore'].quantile(0.25):.2f}")
print(f"  Médiane: {y['MathScore'].median():.2f}")
print(f"  75%: {y['MathScore'].quantile(0.75):.2f}")
print(f"  Max: {y['MathScore'].max():.2f}")
print(f"\nValeurs nulles: {(y['MathScore'] == 0).sum():,} ({(y['MathScore'] == 0).sum()/len(y)*100:.1f}%)")

=== VARIABLE CIBLE: MATHSCORE ===

          MathScore
count  1.172086e+06
mean   9.999513e+01
std    1.221761e+02
min    0.000000e+00
25%    0.000000e+00
50%    6.604880e+01
75%    1.528073e+02
max    8.149725e+02

Distribution:
  Min: 0.00
  25%: 0.00
  Médiane: 66.05
  75%: 152.81
  Max: 814.97

Valeurs nulles: 446,778 (38.1%)


## 3.  Identification des Groupes de Colonnes

In [41]:
# Identifier les différents groupes de colonnes
print("=== GROUPES DE COLONNES ===\n")

# Scores math par question
math_score_cols = [col for col in df.columns if 'math_q' in col and 'average_score' in col]
print(f"✓ Scores math: {len(math_score_cols)} colonnes")

# Scores lecture par question
reading_score_cols = [col for col in df.columns if 'reading_q' in col and 'average_score' in col]
print(f"✓ Scores lecture: {len(reading_score_cols)} colonnes")

# Scores science par question
science_score_cols = [col for col in df.columns if 'science_q' in col and 'average_score' in col]
print(f"✓ Scores science: {len(science_score_cols)} colonnes")

# Timings
math_timing_cols = [col for col in df.columns if 'math_q' in col and 'total_timing' in col]
reading_timing_cols = [col for col in df.columns if 'reading_q' in col and 'total_timing' in col]
science_timing_cols = [col for col in df.columns if 'science_q' in col and 'total_timing' in col]
print(f"\n✓ Timings math: {len(math_timing_cols)} colonnes")
print(f"✓ Timings lecture: {len(reading_timing_cols)} colonnes")
print(f"✓ Timings science: {len(science_timing_cols)} colonnes")

# Variables contextuelles
context_cols = [col for col in df.columns if not any(x in col for x in ['_q', 'ID', 'Unnamed'])]
print(f"\n✓ Variables contextuelles: {len(context_cols)} colonnes")

=== GROUPES DE COLONNES ===

✓ Scores math: 21 colonnes
✓ Scores lecture: 15 colonnes
✓ Scores science: 19 colonnes

✓ Timings math: 21 colonnes
✓ Timings lecture: 15 colonnes
✓ Timings science: 19 colonnes

✓ Variables contextuelles: 193 colonnes


In [42]:
# Analyser les scores math individuels (TRÈS IMPORTANTS pour prédire le score global)
print("=== DISPONIBILITÉ DES SCORES MATH PAR QUESTION ===\n")
math_scores = df[math_score_cols]
math_missing = (math_scores.isna().sum() / len(df) * 100).sort_values()
print(math_missing)
print(f"\nColonnes avec <50% de NaN: {(math_missing < 50).sum()}")

=== DISPONIBILITÉ DES SCORES MATH PAR QUESTION ===

math_q1_average_score      37.228582
math_q2_average_score      37.389918
math_q3_average_score      37.665837
math_q4_average_score      38.580872
math_q5_average_score      39.952273
math_q6_average_score      41.824320
math_q7_average_score      43.334363
math_q8_average_score      45.921289
math_q9_average_score      48.292105
math_q10_average_score     54.699996
math_q11_average_score     62.225297
math_q12_average_score     74.093795
math_q13_average_score     84.687642
math_q14_average_score     92.599775
math_q15_average_score     97.186896
math_q16_average_score     98.124199
math_q17_average_score     99.999915
math_q18_average_score    100.000000
math_q19_average_score    100.000000
math_q20_average_score    100.000000
math_q21_average_score    100.000000
dtype: float64

Colonnes avec <50% de NaN: 9


In [43]:
# Analyser variables démographiques importantes
print("=== VARIABLES DÉMOGRAPHIQUES CLÉS ===\n")
demo_vars = ['AGE', 'GRADE', 'ST004D01T', 'IMMIG', 'OECD', 'CNT']
for var in demo_vars:
    if var in df.columns:
        na_rate = df[var].isna().sum() / len(df) * 100
        print(f"  {var}: {na_rate:.1f}% NaN")

=== VARIABLES DÉMOGRAPHIQUES CLÉS ===

  AGE: 0.3% NaN
  GRADE: 31.3% NaN
  ST004D01T: 0.0% NaN
  IMMIG: 39.4% NaN
  OECD: 0.0% NaN
  CNT: 0.0% NaN


## 4.  Analyse de Corrélation avec MathScore

In [44]:
# Fusionner X et y pour calculer les corrélations
df_with_target = df.merge(y, left_on='Unnamed: 0', right_index=True, how='inner')
print(f"Dataset fusionné: {df_with_target.shape}")

Dataset fusionné: (1172086, 308)


In [45]:
# Calculer les corrélations avec MathScore
numeric_cols = df_with_target.select_dtypes(include=[np.number]).columns
correlations = df_with_target[numeric_cols].corrwith(df_with_target['MathScore']).abs().sort_values(ascending=False)

print("=== TOP 20 COLONNES CORRÉLÉES AVEC MATHSCORE ===\n")
top_corr = correlations.drop('MathScore').head(20)
for i, (col, corr) in enumerate(top_corr.items(), 1):
    print(f"{i:2d}. {col:40s} {corr:.3f}")

/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


=== TOP 20 COLONNES CORRÉLÉES AVEC MATHSCORE ===

 1. reading_q15_average_score                1.000
 2. reading_q15_total_timing                 0.797
 3. reading_q14_total_timing                 0.731
 4. math_q16_average_score                   0.597
 5. math_q15_average_score                   0.534
 6. math_q2_average_score                    0.438
 7. math_q3_average_score                    0.421
 8. math_q14_average_score                   0.415
 9. math_q12_average_score                   0.413
10. math_q13_average_score                   0.407
11. math_q11_average_score                   0.397
12. math_q7_average_score                    0.384
13. math_q1_average_score                    0.375
14. math_q9_average_score                    0.357
15. math_q8_average_score                    0.351
16. math_q10_average_score                   0.348
17. math_q6_average_score                    0.335
18. math_q5_average_score                    0.333
19. ST253                       

## 5. ✂️ Sélection des Colonnes Pertinentes

In [46]:
# Créer la liste des colonnes à conserver
selected_columns = []

# 1. Identifiant (pour le merge)
selected_columns.append('Unnamed: 0')

# 2. Variables démographiques (faible taux de NaN et pertinentes)
selected_columns.extend(['AGE', 'ST004D01T', 'OECD', 'CNT'])

# 3. Scores math par question (les 9 premières avec <50% NaN)
math_scores_to_keep = [f'math_q{i}_average_score' for i in range(1, 10)]
selected_columns.extend(math_scores_to_keep)

# 4. Timing des questions math
math_timing_to_keep = [f'math_q{i}_total_timing' for i in range(1, 10)]
selected_columns.extend(math_timing_to_keep)

# 5. Top scores reading
reading_to_keep = [f'reading_q{i}_average_score' for i in [1, 2, 3, 4, 5, 6]]
selected_columns.extend(reading_to_keep)

# 6. Top scores science
science_to_keep = [f'science_q{i}_average_score' for i in [1, 2, 3, 4, 5, 6]]
selected_columns.extend(science_to_keep)

# 7. Variables contextuelles très corrélées (>0.2)
context_vars = ['ST253', 'ST290', 'ST256', 'ST289', 'IC183', 'ST268', 
                'IC172', 'IC176', 'IC180', 'IC182']
selected_columns.extend(context_vars)

# 8. Variables d'effort
selected_columns.extend(['EFFORT1', 'EFFORT2'])

print(f"✓ {len(selected_columns)} colonnes sélectionnées")
print(f"\nRéduction: {df.shape[1]} → {len(selected_columns)} colonnes ({len(selected_columns)/df.shape[1]*100:.1f}%)")

✓ 47 colonnes sélectionnées

Réduction: 307 → 47 colonnes (15.3%)


In [47]:
# Créer le dataset réduit
df_reduced = df[selected_columns].copy()

print("=== DATASET RÉDUIT ===\n")
print(f"Shape: {df_reduced.shape}")
print(f"Taille mémoire: {df_reduced.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"Réduction: {(1 - df_reduced.memory_usage(deep=True).sum() / df.memory_usage(deep=True).sum()) * 100:.1f}%")

=== DATASET RÉDUIT ===

Shape: (1172086, 47)
Taille mémoire: 469.5 MB
Réduction: 83.8%


## 6. Nettoyage des Données

In [48]:
# Étape 1: Supprimer les lignes avec trop de valeurs manquantes (>70%)
row_nan_percent = (df_reduced.isna().sum(axis=1) / df_reduced.shape[1] * 100)
threshold = 70
rows_to_keep = row_nan_percent <= threshold

df_cleaned = df_reduced[rows_to_keep].copy()

print("=== NETTOYAGE ÉTAPE 1: SUPPRESSION LIGNES AVEC >70% NaN ===\n")
print(f"Lignes conservées: {df_cleaned.shape[0]:,} / {df_reduced.shape[0]:,}")
print(f"Lignes supprimées: {df_reduced.shape[0] - df_cleaned.shape[0]:,} ({(df_reduced.shape[0] - df_cleaned.shape[0])/df_reduced.shape[0]*100:.1f}%)")

=== NETTOYAGE ÉTAPE 1: SUPPRESSION LIGNES AVEC >70% NaN ===

Lignes conservées: 1,053,344 / 1,172,086
Lignes supprimées: 118,742 (10.1%)


In [49]:
# Étape 2: Imputation des valeurs manquantes
print("=== NETTOYAGE ÉTAPE 2: IMPUTATION ===\n")

# Scores: imputation par 0
score_cols = [col for col in df_cleaned.columns if 'average_score' in col ]
for col in score_cols:
    na_count = df_cleaned[col].isna().sum()
    if na_count > 0:
        df_cleaned[col].fillna(0, inplace=True)
        
# timing : imputation par la median
timing_cols = [col for col in df_cleaned.columns if 'total_timing' in col ]
for col in timing_cols:
    na_count = df_cleaned[col].isna().sum()
    if na_count > 0:
        median_val = df_cleaned[col].median()
        df_cleaned[col].fillna(median_val, inplace=True)
        
        

# Variables numériques: imputation par la médiane
numeric_context = ['AGE', 'ST004D01T', 'EFFORT1', 'EFFORT2', 'ST253', 'ST290', 
                   'ST256', 'ST289', 'IC183', 'ST268', 'IC172', 'IC176', 'IC180', 'IC182']
for col in numeric_context:
    if col in df_cleaned.columns:
        na_count = df_cleaned[col].isna().sum()
        if na_count > 0:
            median_val = df_cleaned[col].median()
            df_cleaned[col].fillna(median_val, inplace=True)
        

# Variables catégorielles: imputation par le mode
if 'CNT' in df_cleaned.columns:
    na_count = df_cleaned['CNT'].isna().sum()
    if na_count > 0:
        mode_val = df_cleaned['CNT'].mode()[0]
        df_cleaned['CNT'].fillna(mode_val, inplace=True)


=== NETTOYAGE ÉTAPE 2: IMPUTATION ===



/var/folders/g4/jkwhzpk508sc3j20vhzssdv80000gn/T/ipykernel_32657/4286196222.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned[col].fillna(0, inplace=True)
/var/folders/g4/jkwhzpk508sc3j20vhzssdv80000gn/T/ipykernel_32657/4286196222.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behav

## 7.  Nettoyage des Données

### One hot encoding on CNT

In [50]:
df_cleaned["CNT"].value_counts()

CNT
ESP    42375
CAN    37329
ARE    36114
AUS    25695
BRA    24572
       ...  
QMR     1042
QAR     1006
QUE      942
QUD      900
QUC      788
Name: count, Length: 98, dtype: int64

In [51]:
df_cleaned = pd.get_dummies(df_cleaned, columns=["CNT"], drop_first=True).astype('int')

In [52]:
# === ALIGNER X ET y APRÈS NETTOYAGE ===

# Utiliser l'ID comme index côté X
df_cleaned = df_cleaned.set_index('Unnamed: 0')

# Garder seulement les mêmes élèves dans y
y_aligned = y.loc[df_cleaned.index].copy()

print("Shapes après alignement :")
print("X :", df_cleaned.shape)
print("y :", y_aligned.shape)

Shapes après alignement :
X : (1053344, 142)
y : (1053344, 1)


In [53]:
df_fe = df_cleaned.copy()

math_score_cols = [c for c in df_fe.columns 
                   if c.startswith("math_q") and "average_score" in c]

reading_score_cols = [c for c in df_fe.columns 
                      if c.startswith("reading_q") and "average_score" in c]

science_score_cols = [c for c in df_fe.columns 
                      if c.startswith("science_q") and "average_score" in c]

math_time_cols = [c for c in df_fe.columns 
                  if c.startswith("math_q") and "total_timing" in c]

In [54]:
# MATH
df_fe["math_mean_score"] = df_fe[math_score_cols].mean(axis=1)
df_fe["math_sum_score"]  = df_fe[math_score_cols].sum(axis=1)
df_fe["math_n_answered"] = (df_fe[math_score_cols] > 0).sum(axis=1)

# READING
df_fe["reading_mean_score"] = df_fe[reading_score_cols].mean(axis=1)
df_fe["reading_sum_score"]  = df_fe[reading_score_cols].sum(axis=1)
df_fe["reading_n_answered"] = (df_fe[reading_score_cols] > 0).sum(axis=1)

# SCIENCE
df_fe["science_mean_score"] = df_fe[science_score_cols].mean(axis=1)
df_fe["science_sum_score"]  = df_fe[science_score_cols].sum(axis=1)
df_fe["science_n_answered"] = (df_fe[science_score_cols] > 0).sum(axis=1)


In [55]:
# Time summaries for math
df_fe["math_total_time"] = df_fe[math_time_cols].sum(axis=1)
df_fe["math_mean_time"]  = df_fe[math_time_cols].mean(axis=1)

# Eviter division par zéro
eps = 1e-6
df_fe["math_time_per_point"] = df_fe["math_total_time"] / (df_fe["math_sum_score"] + eps)

df_model = df_fe  # garder tout


In [56]:
from sklearn.model_selection import train_test_split

X = df_model
y_final = y_aligned["MathScore"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y_final, test_size=0.2, random_state=42
)

print(X_train.shape, X_test.shape)


(842675, 154) (210669, 154)


In [59]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

rf = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
r2   = r2_score(y_test, y_pred)

print("RandomForest RMSE:", rmse)
print("RandomForest R²  :", r2)


KeyboardInterrupt: 

## 9. 🚀 Prochaines Étapes

### Feature Engineering
- Créer des features agrégées (moyenne des scores, ratios temps/score)
- Encoder les variables catégorielles (One-Hot Encoding pour CNT)
- Créer des interactions entre variables

### Modélisation
Modèles recommandés:
- **Random Forest** - Robuste et performant
- **XGBoost / LightGBM** - Excellents pour ce type de données
- **Régression linéaire** - Baseline simple

### Validation
- Split train/validation (80/20)
- Cross-Validation (5-fold)
- Métriques: RMSE, MAE, R²

### Optimisation
- GridSearchCV pour le tuning des hyperparamètres
- Feature importance analysis
- Élimination des features peu importantes